In [72]:
import pandas as pd
import numpy as np
import math

def foldChange(diseased, control):
    return diseased/control

def log2foldChange(foldChange):
    return math.log(foldChange, 2)

dat = pd.read_table('test.txt', sep='\t')

controls = []
parkinsons = []
columns = list(dat)
for column in columns:
    if column.startswith('C_'):
        controls.append(column)
    elif column.startswith('P_'):
        parkinsons.append(column)

baseMean = []
contExpMeans = []
parExpMeans = []
foldChanges = []
log2foldChanges = []

for i in range(len(dat.index)):
    mean = np.mean(dat.ix[i,2:])
    contExpMean = np.mean(dat[controls].ix[i,:])
    parExpMean = np.mean(dat[parkinsons].ix[i,:])
    foldCh = foldChange(parExpMean, contExpMean)
    log2foldCh = log2foldChange(foldCh)
    log2foldChanges.append(log2foldCh)
    foldChanges.append(foldCh)
    parExpMeans.append(parExpMean)
    contExpMeans.append(contExpMean)
    baseMean.append(mean)

    

out = {'EnsemblID':dat['EnsemblID'],
       'genes': dat['symbol'],
      'baseMean': baseMean,
      'contExpMean': contExpMeans,
      'parExpMean': parExpMeans,
      'foldCh': foldChanges,
      'log2fCh': log2foldChanges}

outDF = pd.DataFrame(out, columns=['EnsemblID', 'genes', 'baseMean', 'contExpMean', 'parExpMean', 'foldCh', 'log2fCh'])

print(outDF)

            EnsemblID   genes    baseMean  contExpMean  parExpMean    foldCh  \
0  ENSG00000000003.10  TSPAN6  350.414865   325.515392  388.193376  1.192550   
1   ENSG00000000005.5    TNMD    4.589007     5.546770    3.135849  0.565347   
2   ENSG00000000419.8    DPM1  696.910336   740.928471  630.124199  0.850452   

    log2fCh  
0  0.254050  
1 -0.822792  
2 -0.233698  


In [54]:
print()

0    TSPAN6
1      TNMD
2      DPM1
Name: symbol, dtype: object
